In [1]:
import itertools
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.community import k_clique_communities

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
hosp = pd.read_csv('../Data(net)/hosp_sample.csv')
doc = pd.read_csv('../Data(net)/doc_sample.csv')
doc_net = pd.read_csv('../Data(net)/doc_network.csv')

In [3]:
def common_doc_names(Graph, node1_id, node2_id):
    name_list = []
    id_list = Graph.edge[node1_id][node2_id]['common_doc_ids']
    for ids in id_list:
        name_list.append(doc.iloc[ids,1])
    return name_list

In [4]:
# Graph G2 with nodes as hospitals and edges as doctors
G2 = nx.Graph()
G2.name = 'G2 (Node:Hospital, Edge:Doctors)'

In [5]:
#Creating Nodes

nodes_attr = []
for row in range(len(hosp)):
    attr = list(zip(hosp.columns[1:5], hosp.iloc[row,1:5]))
    attr = dict(attr)
    nodes_attr.append(attr)

nodes = list(zip(hosp['HOSP_ID'], nodes_attr))
G2.add_nodes_from(nodes)


In [6]:
#Creating Edges
for i,group in doc_net.groupby('DOC_ID')['HOSP_ID']:
    for u,v in itertools.combinations(group,2):
        set_u = set(doc_net[doc_net['HOSP_ID'] == u].DOC_ID)
        set_v = set(doc_net[doc_net['HOSP_ID'] == v].DOC_ID)
        common_doc_ids = set_u.intersection(set_v)
        G2.add_edge(u, v, {'common_doc_ids':list(common_doc_ids), 'weight':len(common_doc_ids)})


In [7]:
# sorted(G2.edges(data=True), key=lambda x:x[2]['weight'], reverse=True)[:5]

In [8]:
# nx.write_weighted_edgelist(G2, 'G2.edges')

In [9]:
G2.node[1]

{'HOSPITAL NAME': 'Max Super Speciality Hospital',
 'LOCATION': 'Saket, Delhi',
 'NO. OF BEDS': 250,
 'NO. OF DOCTORS': 259}

## Analysis

### Visualization of the Hospital Network
    -Number on the nodes represent Ids of the hospitals.
    -More dense the edge more no. of doctors are being shared between those two nodes.
    -High size of node represents the high betweenness of that node.
    -Different color represents different Communities or rather the location of the hospitals eg.
        -violet ones mostly are from delhi
        -green ones are from bangalore and other south areas

<center>
    <img src="images/G2.png">
</center>

                                                                                              using _Gephi0.91_

In [10]:
# Graph Summary
print(nx.info(G2))

Name: G2 (Node:Hospital, Edge:Doctors)
Type: Graph
Number of nodes: 141
Number of edges: 379
Average degree:   5.3759


#### 1. Degree Centrality- 
The degree of a node is the number of other nodes to which it is connected.
$$ {C_D (u)} = \frac{deg(u)}{{n-1}} $$

In [11]:
# top 5 hospitals sorted according to their degrees
sorted(G2.degree().items(), key=lambda x:x[1], reverse=True)[:5]   # ex. (17,15) 17 is hosp_id and 35 is its degree

[(17, 35), (2, 23), (1, 18), (3, 18), (11, 18)]

In [12]:
degree_centrality = nx.degree_centrality(G2)
print(sorted(degree_centrality.items(), key=lambda x:x[1], reverse=True)[:5])
G2.node[17]

[(17, 0.25), (2, 0.16428571428571428), (1, 0.12857142857142856), (3, 0.12857142857142856), (11, 0.12857142857142856)]


{'HOSPITAL NAME': 'Fortis Hospital',
 'LOCATION': 'Mulund West, Mumbai',
 'NO. OF BEDS': 261,
 'NO. OF DOCTORS': 117}

Fortis Hospital Mumbai is the most connected hospital of all , it is connected to $25\%$ of other hospitals.

#### 2. Closeness Centrality
Closeness Centrality measures how many "hops" it would take to reach every other node in a network (taking the shortest path). It can be informally thought as 'average distance' to all other nodes.

$$ C_C (u) = \frac{n - 1}{\sum_{v=1}^{n-1} d(v, u)} $$

In [13]:
closeness_centrality = nx.closeness_centrality(G2)
sorted(closeness_centrality.items(), key=lambda x:x[1], reverse=True)[:5]    #top5 

[(17, 0.4062248995983936),
 (2, 0.38314393939393937),
 (1, 0.36916058394160584),
 (3, 0.3651624548736462),
 (0, 0.35)]

Averge distance of hospa from every other hospital is simply $\frac{1}{C_C (a)}$

In [14]:
1/closeness_centrality[17]

2.4616905585763718

#### 3. Betweenness Centrality
Betweenness centrality quantifies the number of times a node acts as a bridge (or "broker") along the shortest path between two other nodes.

In this conception, vertices that have a high probability to occur on a randomly chosen shortest path between two randomly chosen vertices have a high betweenness.

$$ C_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)} $$

where ${\sigma(s, t)}$ is total number of shortest paths from node ${s}$ to node ${t}$ and ${\sigma(s, t|v)}$ is the number of those paths that pass through ${v}$.

In [15]:
betweeness_centrality = nx.betweenness_centrality(G2)
sorted(betweeness_centrality.items(), key=lambda x:x[1], reverse=True)[:5]    #top5 

[(17, 0.15776707748374744),
 (2, 0.1066610436817018),
 (4, 0.08501768870258668),
 (25, 0.07427354521904563),
 (11, 0.058137960038108415)]

#### 4. Eigenvector Centrality
A node is high in eigenvector centrality if it is connected to many other nodes who are themselves well connected.

In [16]:
eigenvector_centrality = nx.eigenvector_centrality_numpy(G2)
max_value = max(eigenvector_centrality.items(), key=lambda x: x[1])
print(max_value)

(11, 0.5435779396925311)


In [17]:
ec_scaled = {}
for k in eigenvector_centrality.keys():
    ec_scaled[k] = eigenvector_centrality[k] / max_value[1]

# Scaled by the most central hospital Max Superspeciality Hospital
print(sorted(ec_scaled.items(), key=lambda x:x[1], reverse=True)[0:5])

G2.node[11]

[(11, 1.0), (20, 0.9824094753506462), (18, 0.5201068441382659), (1, 0.5157015885414332), (17, 0.3577478261340102)]


{'HOSPITAL NAME': 'Max Superspeciality Hospital',
 'LOCATION': 'Shalimar Bagh, Delhi',
 'NO. OF BEDS': 300,
 'NO. OF DOCTORS': 135}

In [18]:
no_of_connected_hosps = len(G2) # every other hospital is connected to atleast one hospital.
no_of_connected_hosps

141

Max Superspeciality Hospital is the most central of all of the hospitals.

### Some Important Coefficients (Davis 2003)

In [79]:
# k is average ties of graph per node (interlock per node, Davis(2003))
k = len(G2.edges()) / len(G2.nodes())
k

2.6879432624113475

####  $L_{actual}$ equal the average shortest path length between nodes in the largest connected component. 

In [80]:
larg_conn_comp = max(nx.connected_component_subgraphs(G2), key=len)
L_actual = nx.average_shortest_path_length(larg_conn_comp)
L_actual

len(larg_conn_comp.nodes())

120

#### (DOUBT in L_random!!!!) $L_{equal}$ the average geodesic of the same network in which the random ties (edges) between nodes are random (approximated by $\frac{ln(n)}{ln(k)}$) 

In [81]:
n = len(G2.nodes())
edge_creation_prob = np.log(n)/np.log(k)
#random_larg_conn_comp = nx.fast_gnp_random_graph(n, edge_creation_prob)
#L_random = nx.average_shortest_path_length(random_larg_conn_comp)
L_random = edge_creation_prob
L_random

5.0049336910525986

####  $C_{actual}$ equal the average degree of local clustering in the largest connected component.

In [82]:
C_actual = nx.average_clustering(larg_conn_comp, weight='weight')
C_actual

0.02213462703438445

####  $C_{random}$ is the average degree random of local clustering in the randomized network (approximated by $\frac{k}{n}$).

In [83]:
edge_creation_prob = k/n
#random_larg_conn_comp = nx.fast_gnp_random_graph(n, edge_creation_prob)
#C_random = nx.average_clustering(random_larg_conn_comp, weight='weight')
C_random = edge_creation_prob
C_random

0.01906342739298828

#### Small World Coefficient ($SW$)  (DOUBT-it is coming less than 1 !!!!)

In [84]:
sw = (C_actual/L_actual)*(L_random/C_random)
sw

1.8522532166206298